# Assignment \#6: Asset Pricing

**Due:** Monday 02nd May 2016

Please submit assignments to: 

> `econ370nyu@gmail.com` 

as a Jupyter notebook with your solutions.

In [1]:
%matplotlib inline
import numpy as np

#### Homework problems

** Question \#1 **

Please study the Python function in the following cell.  

Please assume that:

* $P$ is an $n \times n$ matrix of nonnegative numbers whose rows sum to 1

* $x$ is an $n \times 1$ vector

* $y$ is another $n \times 1$ vector

**a.** Please explain what the following things mean:

* $x * y$

* $P @ x$

* $cov$

**Answers:**

**x * y**

In Python the `*` operator performs element-by-element multiplication given x and y are $nx1$ vectors. This operation works as the dimension of both vectors is equal. One benefit of Python is operations are straight forward and do not require the programmer to write an explicit loop.

This operation is a component of the `inner product` and are used extensively in basic linear algebra operations.

**P@x**

In Python the `@` operator performs matrix multiplication. 

Given $P$ is a stochastic matrix, these operations are used to compute the conditional expectation of $x_{t+1}$ given $x_t$ today. 

**$cov$**

Covariance is a measure of how two random variables change together.

$$
\begin{align*}
cov(X,Y) &= E[(X - E[X])(Y-E[Y])] \\
&= E[XY] - E[X]E[Y]
\end{align*}
$$

The Numpy library contains a function ``numpy.cov`` which can compute the covariance.


**b.** Please propose some words in English telling what the function does

In [2]:
def covd(P,x,y):
    """
    
    """
    Ex = P @ x
    Ey = P @ y
    Exy = P @ (x*y)
    cov = Exy - Ex*Ey
    return cov

**Answer:** 

This Python function requires 3 inputs:

* $P$ is a $n \times n$ stochastic matrix

* $x$ is an $n \times 1$ vector (Random Variable)

* $y$ is another $n \times 1$ vector (Random Variable)

This function then computes three components of the covariance formulae $E_x$, $E_y$, and $E_{xy}$ using the stochastic matrix $P$ and returns the final computation:

$$
cov = E_{xy} - E_xE_y
$$

-------------------------------------------------------------------------------------------------------------------

** Question \#2 **

In the next cell, we claim to generate a transition matrix 5 state Markov chain

In [3]:
def create_disaster_probability(diag=0.9, alpha=0.01, n=5):
    # Allocate for P
    offdiag = (1 - diag - alpha)/(n-2)
    P = np.ones((n, n)) * offdiag
    
    # Add to diag
    P = P + np.eye(5)*(diag-offdiag)
    
    # Update last column
    P[:, -1] = alpha
    
    # Last row
    P[-1, :] = np.zeros(n)
    P[-1, -1] = 0.5
    P[-1, n//2] = 0.5
    
    return P

P = create_disaster_probability(alpha=0.05)

**Please print out P and verify that it is a legitimate Markov transition matrix**

In [4]:
P

array([[ 0.9       ,  0.01666667,  0.01666667,  0.01666667,  0.05      ],
       [ 0.01666667,  0.9       ,  0.01666667,  0.01666667,  0.05      ],
       [ 0.01666667,  0.01666667,  0.9       ,  0.01666667,  0.05      ],
       [ 0.01666667,  0.01666667,  0.01666667,  0.9       ,  0.05      ],
       [ 0.        ,  0.        ,  0.5       ,  0.        ,  0.5       ]])

In [5]:
#-All Rows Sum to 1-#
P.sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.])

In [6]:
#-Elements are positive-#
P[P < 0]

array([], dtype=float64)

--------------------------------------------------------------------------------------------------------------------

**Question \#3**
 
 
A friend tells you that the price of an asset satisfies

$$ p_t = E_t m_{t+1} E_t (c_{t+1} + p_{t+1}) + cov_t (m_{t+1}, c_{t+1} + p_{t+1}) \quad (1) $$

where $p_t = v_t c_t$

* $v_t$ is a function of the Markov state at $t$

and 

* $E_t x_{t+1}$ is the conditional expectation of $x_{t+1}$
* $m_t$ is a function of the Markov state 
* $s_t$ is the Markov state
* $P$ matrix is from Question \#2 
 

Three vectors m, v, and s are given by the successive **rows** of

* $m=[0.45351474,0.4759072,0.5,0.52596976,2.]$,
* $v=[1.11737826,1.15554868,1.19713735,1.24262496,2.09856868]$
* $s=[1.05,1.025,1.,0.975,0.5]$

Please interpret these three vectors as random variables.  

Please **verify** that equation (1) holds with these values. (You'll have to write Python code to do this.)

In [7]:
#-Setup the problem as Numpy Arrays-#
m = np.array([0.45351474,0.4759072,0.5,0.52596976,2.])
v = np.array([1.11737826,1.15554868,1.19713735,1.24262496,2.09856868])
s = np.array([1.05,1.025,1.,0.975,0.5])

**Solution Strategy \#1**

We know the definition of covariance is:

$$
E_t (x_{t+1} y_{t+1}) = {\rm cov}_t (x_{t+1}, y_{t+1}) + E_t x_{t+1} E_t y_{t+1}
$$

the price equation can be rearranged 

$$
p_t = E_t[m_{t+1}(p_{t+1} + c_{t+1})]
$$

Your friend has told you to guess: $p_t = v_t * c_t$ where

* $v_t$ is a function of the Markov state at $t$

Substituting into the equation one can cancel $c_t$ as $c_{t+1} = \lambda_{t+1} c_t$ where $\lambda_{t+1}$ is a function of the state $s$. Solving the equation for $v_t$

$$
v_t = E_t[m_{t+1}(v_{t+1}\lambda_{t+1} + \lambda_{t+1})]
$$

* where $\lambda$ is a function of the markov state $s$

This equation is now expressed in terms of $m$, $v$, and $s$ which is what we were given. 

In [8]:
#-Compute the Expression using Linear Algebra-#
exp = m * (v * s + s)
# m is a vector and *, and + operators are element by element

In [9]:
#-Compute the Expectation-#
E_exp = P @ exp

In [10]:
print(E_exp)

[ 1.11737826  1.15554868  1.19713735  1.24262497  2.09856868]


In [11]:
print(v)

[ 1.11737826  1.15554868  1.19713735  1.24262496  2.09856868]


The expression **holds** as:

$$
v = E\_exp
$$


In [12]:
np.allclose(E_exp,v)

True

**Solution Strategy \#2**

Given

$$ 
p_t = E_t m_{t+1} E_t (c_{t+1} + p_{t+1}) + cov_t (m_{t+1}, c_{t+1} + p_{t+1}) \quad (1) 
$$

where $p_t = v_t c_t$

* $v_t$ is a function of the Markov state at $t$

Substituting into the expression for $p_t$ you can cancel $c_t$ terms as $c_{t+1} = \lambda_{t+1} c_t$. This evaluates to:

$$
v_t = E_t m_{t+1} E_t (v_{t+1} \lambda_{t+1} + \lambda_{t+1}) + cov_t (m_{t+1}, v_{t+1} \lambda_{t+1} + \lambda_{t+1})
$$

You need to use the covariance rule:

$$
COV(aX,bY) = axCOV(X,y)
$$

We can now compute all elements of this expression using $m$, $v$, and $s$ by computing Expectations

In [13]:
E_m = P @ m
print(E_m)

[ 0.53319455  0.55297456  0.57425653  0.59719648  1.25      ]


In [14]:
E_s = P @ s
print(E_s)

[ 1.02        0.99791667  0.97583333  0.95375     0.75      ]


In [15]:
E_vs = P @ (v*s)
print(E_vs)

[ 1.16827224  1.17815694  1.18937523  1.20211465  1.12321085]


In [16]:
cov = covd(P,m,v*s+s)
print(cov)

[-0.04939657 -0.04776465 -0.0462478  -0.04484982 -0.24294488]


In [17]:
ans = E_m*(E_vs+ E_s) + cov
print("ans/v = 1?", ans/v)

ans/v = 1? [ 1.          1.          1.          1.00000001  1.        ]


In [18]:
ans

array([ 1.11737826,  1.15554868,  1.19713735,  1.24262497,  2.09856868])

In [19]:
v

array([ 1.11737826,  1.15554868,  1.19713735,  1.24262496,  2.09856868])

In [20]:
np.allclose(ans,v)

True

-------------------------------------------------------------------------------------------------------------------

**Question \#4**

#### Returns

Returns are defined as $R_{t+1} = \frac{v_{t+1} + s_{t+1}}{v_t}$.

In our Markov setting, returns can be summarized by an $n \times n$ matrix whose $i,j$th component
is 

$$ R_{i,j} = \frac{(v_j s_j + s_j)}{v_i} $$

**Part A** Please create the matrix $R$ and verify

$$ 1 = E_t m_{t+1} E_t R_{t+1} + cov_t (m_{t+1}, R_{t+1}) $$

** Solution \#1 to Creating the R Matrix **

In [21]:
#-Compute Numerator as an array-#
Rnum = (v*s+s)

In [22]:
Rnum

array([ 2.22324717,  2.2094374 ,  2.19713735,  2.18655934,  1.54928434])

In [23]:
#-Compute Denominator as an array-#
Rden = 1/v

In [24]:
Rden

array([ 0.89495208,  0.86538976,  0.83532604,  0.80474804,  0.47651526])

Check the definition of of R

$$ R_{i,j} = \frac{(v_j s_j + s_j)}{v_i} $$

Numerator are ``j`` and denominator is ``i``. Therefore for all ``j`` we need to devide by each ``i``. This can be done row by row, looping, or making use of numpy.

In [25]:
Rden.reshape(5,1)

array([[ 0.89495208],
       [ 0.86538976],
       [ 0.83532604],
       [ 0.80474804],
       [ 0.47651526]])

In [26]:
R = Rnum * Rden.reshape(5,1)

In [27]:
R

array([[ 1.98969969,  1.9773406 ,  1.96633265,  1.95686583,  1.38653525],
       [ 1.92397535,  1.91202451,  1.90138017,  1.89222607,  1.34073481],
       [ 1.85713625,  1.84560059,  1.83532604,  1.82648995,  1.29415755],
       [ 1.7891538 ,  1.77804041,  1.76814197,  1.75962934,  1.24678353],
       [ 1.0594112 ,  1.05283064,  1.04696948,  1.04192889,  0.73825763]])

**Verify**

$$ 1 = E_t m_{t+1} E_t R_{t+1} + cov_t (m_{t+1}, R_{t+1}) $$

In [28]:
Etm = P @ m
EtR = np.sum(P*R, axis=1)
EtRm = np.sum(m*P*R, axis=1)
cov = EtRm - Etm * EtR

In [29]:
exp = Etm * EtR + cov

In [30]:
exp

array([ 1.        ,  1.        ,  1.        ,  1.00000001,  1.        ])

In [31]:
ans = np.array([1]*5)

In [32]:
ans

array([1, 1, 1, 1, 1])

In [33]:
np.allclose(ans, exp)

True

#### Solve using linear algebra rather than using NumPy Vector Operations

In [34]:
#-This reult is computed using (P*R).sum(axis=1)
EtR

array([ 1.9583988 ,  1.88315182,  1.80865509,  1.73492785,  0.89261355])

In [35]:
#-Equivalent using Matrix Multiplication-#
np.diag(P @ R.T)

array([ 1.9583988 ,  1.88315182,  1.80865509,  1.73492785,  0.89261355])

In [36]:
#-Take a look at what is being computed here-#
P @ R.T 

array([[ 1.9583988 ,  1.8937084 ,  1.82792078,  1.76100779,  1.04274512],
       [ 1.9474816 ,  1.88315182,  1.81773094,  1.75119097,  1.03693228],
       [ 1.93775791,  1.87374933,  1.80865509,  1.74244735,  1.03175492],
       [ 1.92939556,  1.8656632 ,  1.80084988,  1.73492785,  1.0273024 ],
       [ 1.67643395,  1.62105749,  1.56474179,  1.50746275,  0.89261355]])

This matrix operation computes all combinations of rows and columns, as per the rules of Matrix multiplication. What we are interested in, is the the diagonal elements as they represent the sum of each row of $P$ multiplied element-by-element with the `rows` of R given R is transposed.

#### Possibly helpful or harmful hint

The following code is one person's attempt to answer this.
Do you think this code is correct?
If can you do better?

Please explain

In [37]:
# Here is some code that implements the pseudo code R = (v*s + s)(tomorrow)/v(today)

v=np.array([ 1.11737826,  1.15554868,  1.19713735,  1.24262496,  2.09856868])
m=np.array([ 0.45351474,  0.4759072 ,  0.5       ,  0.52596976,  2.        ])
s=np.array([ 1.05 ,  1.025,  1.   ,  0.975,  0.5  ])

R1 = np.array((v*s + s)/v[0])
R2 = np.array((v*s + s)/v[1])
R3 = np.array((v*s + s)/v[2])
R4 = np.array((v*s + s)/v[3])
R5 = np.array((v*s + s)/v[4])

R = np.vstack([R1,R2,R3,R4,R5])
R

array([[ 1.98969969,  1.9773406 ,  1.96633265,  1.95686583,  1.38653525],
       [ 1.92397535,  1.91202451,  1.90138017,  1.89222607,  1.34073481],
       [ 1.85713625,  1.84560059,  1.83532604,  1.82648995,  1.29415755],
       [ 1.7891538 ,  1.77804041,  1.76814197,  1.75962934,  1.24678353],
       [ 1.0594112 ,  1.05283064,  1.04696948,  1.04192889,  0.73825763]])

**Explanation:**

This code block redefines $v$, $m$, and $s$ and then constructs the $R$ matrix using NumPy broadcasting over the 5 different states. $R1$ to $R5$ correspond to the 5 different states of the problem. The vector expression $v_j*s_j + s_j$ produces a $1 \times 5$ vector of which each element is normalised by $v_i$ for each state $i$. This correspondes with the formal definition of the $R$ matrix:

$$ R_{i,j} = \frac{(v_j s_j + s_j)}{v_i} $$

The $R$ matrix is then fully constructed by stacking each row of the matrix

**Part C**

An unnamed student in our class has said that she thinks the following code does
the same thing and is "better". Do you agree?  Can you please explain how this code works?


In [38]:
Rsf = np.reshape(v*s + s, (1, 5)) / np.reshape(v, (5, 1))

In [39]:
Rsf

array([[ 1.98969969,  1.9773406 ,  1.96633265,  1.95686583,  1.38653525],
       [ 1.92397535,  1.91202451,  1.90138017,  1.89222607,  1.34073481],
       [ 1.85713625,  1.84560059,  1.83532604,  1.82648995,  1.29415755],
       [ 1.7891538 ,  1.77804041,  1.76814197,  1.75962934,  1.24678353],
       [ 1.0594112 ,  1.05283064,  1.04696948,  1.04192889,  0.73825763]])

**Comment** While this code is correct it is actually more confusing than either of the above two solutions. It needs comments to explain what the role of reshape is.

In [40]:
#-The first reshape is unncessary as that is the natural shape of the vector in numpy-#
Rsf = (v*s + s) / np.reshape(v, (5, 1))

In [41]:
Rsf

array([[ 1.98969969,  1.9773406 ,  1.96633265,  1.95686583,  1.38653525],
       [ 1.92397535,  1.91202451,  1.90138017,  1.89222607,  1.34073481],
       [ 1.85713625,  1.84560059,  1.83532604,  1.82648995,  1.29415755],
       [ 1.7891538 ,  1.77804041,  1.76814197,  1.75962934,  1.24678353],
       [ 1.0594112 ,  1.05283064,  1.04696948,  1.04192889,  0.73825763]])

-------------------------------------------------------------------------------------------------------------------

**Question \#5**

Consider the problem in Question \#3

$$ p_t = E_t m_{t+1} E_t (c_{t+1} + p_{t+1}) + cov_t (m_{t+1}, c_{t+1} + p_{t+1}) \quad (1) $$

where $p_t = v_t c_t$

* $v_t$ is a function of the Markov state at $t$

and 
 
* $E_t x_{t+1}$ is the conditional expectation of $x_{t+1}$
*  $m_t$ is a function of the Markov state 
*  $s_t$ isthe Markov state
* $P$ is the same as in Question \#2
 

Suppose you **only** know P, m, and s:

* $m=[ 0.45351474,  0.4759072 ,  0.5       ,  0.52596976,  2.        ]$,
* $s =[ 1.05 ,  1.025,  1.   ,  0.975,  0.5  ]$
    
How do you compute the vector $v$ to satisfy the asset pricing function? 

[**Hint:** Check that your computed v is equal to $v =[ 1.11737826,  1.15554868,  1.19713735,  1.24262496,  2.09856868]$]



**Strategy \#1: Using Iteration to Solve for V**

We Know

$$
v_t = E_t[m_{t+1}(v_{t+1}\lambda_{t+1} + \lambda_{t+1})]
$$

We can guess that V = [0,0,0,0,0] and iterate over the equation until we converge to a solution. 

It is easier to express the vectors as **column** vectors

In [42]:
#-Assume V = [0,0,0,0,0]-#
vcol = np.reshape(np.zeros(5), (5,1))

In [43]:
vcol

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [44]:
mcol = np.reshape(m, (5,1))

In [45]:
mcol

array([[ 0.45351474],
       [ 0.4759072 ],
       [ 0.5       ],
       [ 0.52596976],
       [ 2.        ]])

In [46]:
scol = np.reshape(s, (5,1))

In [47]:
scol

array([[ 1.05 ],
       [ 1.025],
       [ 1.   ],
       [ 0.975],
       [ 0.5  ]])

In [48]:
#-Iterate to Solve for V-#
maxitr = 1000
for i in range(maxitr):
    vcol_new = P @ (mcol * (vcol * scol + scol))
    if np.allclose(vcol, vcol_new, rtol=1e-08):
        break
    else:
        vcol = vcol_new #Iterative Again

In [49]:
vcol

array([[ 1.11737824],
       [ 1.15554866],
       [ 1.19713733],
       [ 1.24262494],
       [ 2.09856861]])

In [50]:
#-Number of Iterations
print(i)

34


**Compare computed V with Known Solution of V as Column Vectors**

In [51]:
#-Known Solution of v-#
np.reshape(v, (5,1))

array([[ 1.11737826],
       [ 1.15554868],
       [ 1.19713735],
       [ 1.24262496],
       [ 2.09856868]])

In [52]:
vcol

array([[ 1.11737824],
       [ 1.15554866],
       [ 1.19713733],
       [ 1.24262494],
       [ 2.09856861]])

In [53]:
np.allclose(np.reshape(v, (5,1)), vcol)

True

**Strategy \#2: Using Linear Algebra**

We know from Question \#3, the pricing equation can be represented as

$$
v_t = E_t[m_{t+1}(v_{t+1}\lambda_{t+1} + \lambda_{t+1})]
$$

where $\lambda_t = s_t$

We know that $v$ is a vector across 5 states. First consider the first state and dropping subscript $t$ for convenience. The above equation would be

$$
v_1 = [P_1,...,P_5]*\begin{bmatrix} m_1(v_1\lambda_1 + \lambda_1) \\ ... \\ m_5(v_5\lambda_5 + \lambda_5) \end{bmatrix}
$$

Now computing for all states requires solving this equation where $P$ is a matrix, $v$, $m$, and $\lambda$ are vectors.

$$
v = P[m(v\lambda + \lambda)]
$$

We will need to use some linear algebra to solve this equation given v is on both sides of the equation. Transform element by element operations ($*$) on $\lambda$ to matrix operations ($@$) by converting the $lambda$ and $m$ vector into diagonal matrices. (**Note** Write out this problem in full matrix notation and verify for yourself) The notation that follows is CAPITAL for Matrix and lower case for a Vector

$$
\Lambda = 
\begin{bmatrix}
s_1 & 0 & 0 & 0 & 0 \\ 
0 & s_2 & 0 & 0 & 0  \\
0 & 0 & s_3 & 0 & 0  \\
0 & 0 & 0 & s_4 & 0  \\
0 & 0 & 0 & 0 &  s_5 \\
\end{bmatrix}
$$

and

$$
M = 
\begin{bmatrix}
m_1 & 0 & 0 & 0 & 0 \\ 
0 & m_2 & 0 & 0 & 0  \\
0 & 0 & m_3 & 0 & 0  \\
0 & 0 & 0 & m_4 & 0  \\
0 & 0 & 0 & 0 & m_5 \\
\end{bmatrix}
$$

Then solving using **rules** of linear algebra

$$
[I-PM\Lambda]v = P M s
$$

Therefore

$$
v = [I-PM\Lambda]^{-1}PM s
$$


In [54]:
I = np.eye(5)
m=np.array([ 0.45351474,  0.4759072 ,  0.5       ,  0.52596976,  2.        ])
s=np.array([ 1.05 ,  1.025,  1.   ,  0.975,  0.5  ])

In [55]:
#-Represented problem as Full Matrices, to use proper Linear Algebra Operations-#
M = np.diag(m)
S = np.diag(s)

In [56]:
Vsolve = np.linalg.inv(I - P @ M @ S) @ P @ (M @ s)

In [57]:
Vsolve

array([ 1.11737826,  1.15554869,  1.19713735,  1.24262497,  2.09856868])

In [58]:
v

array([ 1.11737826,  1.15554868,  1.19713735,  1.24262496,  2.09856868])

In [59]:
np.allclose(Vsolve, v)

True

**Confirmation of RHS == LHS**

$$
[I-PM\Lambda]v = P M s
$$

In [60]:
#-LHS-#
(I - P @ M @ S ) @ v

array([ 0.50358185,  0.51384124,  0.5246136 ,  0.53593838,  0.75      ])

In [61]:
#-RHS-#
P @ M @ s

array([ 0.50358185,  0.51384124,  0.5246136 ,  0.53593839,  0.75      ])